### --- Day 14: Restroom Redoubt ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/14

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 14);
WriteLines(inputLines);

Loading puzzle file: Day14.txt
Total lines: 500
Max line length: 18

p=39,34 v=40,73
p=43,71 v=-56,-11
p=87,9 v=71,41
p=67,40 v=12,6
p=54,54 v=-22,73


In [4]:
string[] testInputLines = [
    "p=0,4 v=3,-3",
    "p=6,3 v=-1,-3",
    "p=10,3 v=-1,2",
    "p=2,0 v=2,-1",
    "p=0,0 v=1,3",
    "p=3,0 v=-2,-2",
    "p=7,6 v=-1,-3",
    "p=3,0 v=-1,-2",
    "p=9,3 v=2,3",
    "p=7,3 v=-1,2",
    "p=2,4 v=2,-3",
    "p=9,5 v=-3,-3",
];

This first part looks fairly straightforward: move by `100 x velocity`, calculate the position modulo room size.

In [5]:
record struct Robot(Point Position, Point Velocity) {}

In [6]:
Robot ParseRobot(string inputLine)
{
    var bits = inputLine.Split(["p=", " v=", ","], StringSplitOptions.RemoveEmptyEntries)
                .Select(int.Parse)
                .ToArray();
    
    var (pX, pY, vX, vY) = bits;
    return new((pX, pY), (vX, vY));
}


In [7]:
// The robots outside the actual bathroom are in a space which is 101 tiles wide
// and 103 tiles tall (when viewed from above). However, in this example, the
// robots are in a space which is only 11 tiles wide and 7 tiles tall.

using Room = (int Width, int Height);

Room testRoom = new(11, 7);
Room part1Room = new(101, 103);

In [8]:
Point Walk(Robot robot, Room room, int seconds)
{
    var newX = WalkOne(robot.Position.X, robot.Velocity.X, seconds, room.Width);
    var newY = WalkOne(robot.Position.Y, robot.Velocity.Y, seconds, room.Height);

    return (newX, newY);
}

int WalkOne(int position, int velocity, int seconds, int size)
{
    var newPos = (position + velocity * seconds) % size;

    newPos = newPos switch {
        < 0 => newPos + size,
        _ => newPos
    };

    return newPos;
}

int GetQuadrant(Point point, Room room) 
{
    var xHalf = room.Width / 2;
    int yHalf = room.Height / 2;

    // Quadrants are 1, 2, 3, 4, clockwise from top-left. 0 means on the border
    // therefore no quadrant
    var quadrant = (point.X - xHalf, point.Y - yHalf) switch {
        (< 0, < 0) => 1,
        (> 0, < 0) => 2,
        (> 0, > 0) => 3,
        (< 0, > 0) => 4,
        _ => 0
    };

    return quadrant;
}

int WalkAll(string[] inputLines, Room room)
{
    const int seconds = 100;

    var x = from inputLine in inputLines
            let robot = ParseRobot(inputLine)
            let newPos = Walk(robot, room, seconds)
            group newPos by GetQuadrant(newPos, room) into quadrants
            select (quadrant: quadrants.Key, count: quadrants.Count());

    return x.Where(qc => qc.quadrant != 0).Select(qc => qc.count).Aggregate(1, (a, b) => a * b);
}

In [9]:
// In this example, the quadrants contain 1, 3, 4, and 1 robot. Multiplying
//these together gives a total safety factor of 12.

var testAnswer = WalkAll(testInputLines, testRoom);
Console.WriteLine(testAnswer);

12


In [10]:
// Predict the motion of the robots in your list within a space which is 101
// tiles wide and 103 tiles tall. What will the safety factor be after exactly 100
// seconds have elapsed?

var part1Answer = WalkAll(inputLines, part1Room);
Console.WriteLine(part1Answer);

211773366


In [11]:
// 211773366 is correct!
Ensure(211773366, part1Answer);